#### train the model on the training set

In [1]:
# positive 1 depressed
# negative 0 non-depressed

# import pickle
import pickle5 as pickle
from model import CNN
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import torchvision.transforms as transforms
import copy

import random, os
import torch
import numpy as np
seed = 32
random.seed(seed)
os.environ['PYTHONHASHSEED'] =str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic =True


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

label_csv = pd.read_csv('../processed_data/train_split_Depression_AVEC2017.csv')
user_list = list(label_csv['Participant_ID'])

with open('../processed_data/train_labels.pickle', 'rb') as handle:
    train_labels = pickle.load(handle)


X = []
y = []
aug_ratio_pos = 32
aug_ratio_neg = 16
top_k_reserve = 5
marked_uttr = 1
# marked_uttr = 0

for user in user_list:
    feature = np.array(np.load('../processed_data/feature_fc1/'+str(user)+'.npy'))
    saliency = list(np.load('../processed_data/saliency/'+str(user)+'.npy'))
    # print(saliency)
    select_range = range(len(saliency) - 10) if len(saliency) != 10 else [0]
    # print(select_range)
    aug_ratio = aug_ratio_pos if train_labels[user] else aug_ratio_neg

    for i in range(aug_ratio):
        start_idx = np.random.choice(select_range, replace=False, size=1)[0]
        feature_part = feature[start_idx:start_idx+10,:]
        # print(feature_part.shape)
        top_idx = np.argsort(saliency[start_idx:start_idx+10])[:top_k_reserve]
        remaining_idx = list(set(range(10)) - set(top_idx))
        selected_idx = np.random.choice(remaining_idx, replace=False, size=marked_uttr)
        if np.random.rand() < 0.5:
            for idx in selected_idx:
                feature_part[idx,:] = 0.001
                # feature_part[idx,:] = np.mean(feature_part[idx,:])
        X.append(torch.Tensor(np.transpose(feature_part)))
        y.append(train_labels[user])
    
tensor_y_train = torch.Tensor(np.array(y)).to(torch.float)

tensor_X_train = torch.stack((X))


trainDataset = TensorDataset(tensor_X_train, tensor_y_train)

trainLoader = DataLoader(trainDataset, batch_size=16, shuffle=True)


cnn = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=0.001, weight_decay=0.01)

for epoch in range(10):
    running_loss = 0.0
    running_acc = 0.0

    with tqdm(trainLoader, unit="batch") as tepoch:

        for data in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}")
            x, y = data
            x = Variable(x).to(device)
            y = Variable(y).to(device)

            out = cnn(x)
            loss = criterion(out, y.long())

            _, pred = torch.max(out, 1)

            num_correct = (pred == y).sum()
            running_acc += num_correct.item()
            running_loss = running_loss + loss.detach()*y.size(0)

            acc = (pred == y).float().mean()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


Epoch 10: 100%|██████████| 138/138 [00:01<00:00, 128.08batch/s]


#### validate the trained model on the validation set
##### we applied a weighted voting based on the length of a window only for window size is 10. This slighly helped with the performance.

In [2]:
sample_length = 10
overlap = 0
label_csv = pd.read_csv('../processed_data/dev_split_Depression_AVEC2017.csv')
user_list = list(label_csv['Participant_ID'])

with open('../processed_data/dev_labels.pickle', 'rb') as handle:
    dev_labels = pickle.load(handle)


X = []
y = []
all_length = []
all_saliency = []
user_names = []
for user in user_list:
    sorted_feature = np.array(np.load('../processed_data/feature_fc1/'+str(user)+'.npy'))
    length = np.load('../processed_data/length/'+str(user)+'.npy')
    saliency = list(np.load('../processed_data/saliency/'+str(user)+'.npy'))

    starting_idx = 0
    x_tmp = []
    saliency_tmp = []
    length_tmp = []
    while starting_idx + sample_length < len(length):
        sorted_feature_tmp = sorted_feature[starting_idx:starting_idx+sample_length]
        sorted_feature_tmp = torch.Tensor(np.transpose(sorted_feature_tmp))
        x_tmp.append(sorted_feature_tmp)
        saliency_tmp.append(np.sum(saliency[starting_idx:starting_idx+sample_length]))
        length_tmp.append(np.sum(length[starting_idx:starting_idx+sample_length]))
        starting_idx = starting_idx + sample_length-overlap

    X.append(torch.stack((x_tmp)))
    y.append(dev_labels[user])

    all_saliency.append(saliency_tmp)
    all_length.append(length_tmp)

    user_names.append(user)
    

weight_ref = all_length
    

cnn.eval()

y_true = []
y_pred = []

for i in range(0, len(X)):
    y_true.append(y[i])
    x = Variable(X[i]).to(device)
    out = F.softmax(cnn(x), dim=1)
    prob_tmp = out.data.cpu().numpy()
    tmp = np.array(weight_ref[i])
    prob_tmp = prob_tmp * tmp.reshape((tmp.size, 1))
    prob_tmp = np.sum(prob_tmp, axis=0)

    prob = prob_tmp / out.shape[0]
    y_pred.append(np.argmax(prob, axis=0))


print(classification_report(y_true, y_pred, zero_division=0))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.75      0.78      0.77        23
           1       0.55      0.50      0.52        12

    accuracy                           0.69        35
   macro avg       0.65      0.64      0.64        35
weighted avg       0.68      0.69      0.68        35

[[18  5]
 [ 6  6]]


#### provide the running environment

In [3]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
import numpy as np
import os
import subprocess
import sys
from collections import defaultdict
import PIL
import torch
import torchvision
from tabulate import tabulate

__all__ = ["collect_env_info"]

def get_env_module():
    var_name = "DETECTRON2_ENV_MODULE"
    return var_name, os.environ.get(var_name, "<not set>")


def collect_torch_env():
    try:
        import torch.__config__

        return torch.__config__.show()
    except ImportError:
        # compatible with older versions of pytorch
        from torch.utils.collect_env import get_pretty_env_info

        return get_pretty_env_info()


def collect_env_info():
    data = []
    data.append(("sys.platform", sys.platform))
    data.append(("Python", sys.version.replace("\n", "")))
    data.append(("Numpy", np.__version__))

    data.append(get_env_module())
    data.append(("PyTorch", torch.__version__))
    data.append(("PyTorch Debug Build", torch.version.debug))
    try:
        data.append(("torchvision", torchvision.__version__))
    except AttributeError:
        data.append(("torchvision", "unknown"))

    has_cuda = torch.cuda.is_available()
    data.append(("CUDA available", has_cuda))
    if has_cuda:
        devices = defaultdict(list)
        for k in range(torch.cuda.device_count()):
            devices[torch.cuda.get_device_name(k)].append(str(k))
        for name, devids in devices.items():
            data.append(("GPU " + ",".join(devids), name))

        from torch.utils.cpp_extension import CUDA_HOME

        data.append(("CUDA_HOME", str(CUDA_HOME)))

        if CUDA_HOME is not None and os.path.isdir(CUDA_HOME):
            try:
                nvcc = os.path.join(CUDA_HOME, "bin", "nvcc")
                nvcc = subprocess.check_output("'{}' -V | tail -n1".format(nvcc), shell=True)
                nvcc = nvcc.decode("utf-8").strip()
            except subprocess.SubprocessError:
                nvcc = "Not Available"
            data.append(("NVCC", nvcc))

        cuda_arch_list = os.environ.get("TORCH_CUDA_ARCH_LIST", None)
        if cuda_arch_list:
            data.append(("TORCH_CUDA_ARCH_LIST", cuda_arch_list))
    data.append(("Pillow", PIL.__version__))

    try:
        import cv2

        data.append(("cv2", cv2.__version__))
    except ImportError:
        pass
    env_str = tabulate(data) + "\n"
    env_str += collect_torch_env()
    return env_str


if __name__ == "__main__":
    print(collect_env_info())

---------------------  --------------------------------------------------
sys.platform           linux
Python                 3.6.9 (default, Jun 29 2022, 11:45:57) [GCC 8.4.0]
Numpy                  1.19.5
DETECTRON2_ENV_MODULE  <not set>
PyTorch                1.10.2+cu113
PyTorch Debug Build    False
torchvision            0.11.3+cu113
CUDA available         True
GPU 0,1                NVIDIA RTX A6000
CUDA_HOME              /usr/local/cuda
NVCC                   Build cuda_11.2.r11.2/compiler.29618528_0
Pillow                 8.4.0
---------------------  --------------------------------------------------
PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: